# Assignment

1. Identify cases during which blood pressure dropped below the norm for the age (see below) for 14 continuous minutes or longer. Assume, that the PERSON_ID is the identifier for the patient, and SERVICE_DATE is the date of the surgery that they had. Surgeries don’t span over 1 day. The Age is given for that patient, for the surgery date.  The Blood pressure is only taken during the surgery duration. 

In [99]:
import os
os.chdir('C:\\Users\\M246047\\Documents\\Python')
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

bp = pd.read_csv('BloodPressure.csv')
bp = pd.DataFrame(bp)

dems = pd.read_csv('Demographics.csv')
dems = pd.DataFrame(dems)

bp.head()

,PERSON_ID,TIME,SYSTOLIC_BLOOD_PRESSURE
0,123,1/5/2016 7:16,33
1,123,1/5/2016 7:17,75
2,123,1/5/2016 7:18,58
3,123,1/5/2016 7:19,93
4,123,1/5/2016 7:20,35


## Scanning the dataframes for impurities and null values

In [100]:
# bp['SYSTOLIC_BLOOD_PRESSURE'] = bp['SYSTOLIC_BLOOD_PRESSURE'].astype(float)
bp.describe()
bp.SYSTOLIC_BLOOD_PRESSURE.value_counts()

88       37
33       36
73       35
44       35
100      34
         ..
79        9
76        9
ERROR     4
28        2
26        1
Name: SYSTOLIC_BLOOD_PRESSURE, Length: 75, dtype: int64

In [101]:
bp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1581 entries, 0 to 1580
Data columns (total 3 columns):
PERSON_ID                  1581 non-null int64
TIME                       1581 non-null object
SYSTOLIC_BLOOD_PRESSURE    1580 non-null object
dtypes: int64(1), object(2)
memory usage: 37.2+ KB


In [102]:
bp.PERSON_ID.value_counts()

456    480
234    360
345    240
123    240
789    120
987     59
567     46
111     36
Name: PERSON_ID, dtype: int64

In [103]:
dems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 3 columns):
PERSON_ID       17 non-null int64
SERVICE_DATE    17 non-null object
AGE_MONTHS      17 non-null int64
dtypes: int64(2), object(1)
memory usage: 536.0+ bytes


In [104]:
dems.PERSON_ID.value_counts()

456    4
234    3
987    2
123    2
345    2
567    2
789    1
111    1
Name: PERSON_ID, dtype: int64

In [105]:
bp.loc[bp['SYSTOLIC_BLOOD_PRESSURE'].isna()]

,PERSON_ID,TIME,SYSTOLIC_BLOOD_PRESSURE
1514,987,10/17/2016 18:40,NaN


## Handling Impure Data

Because there are  one record with a null value and four with ERRORs in the blood pressure variable, I will drop those records. 

I'm going to parse the TIME column in the BloodPressure dataset into two columns: one for SERVICE_DATE to merge with the Demographics file, and the other for TIME.

In [106]:
bd = bp.drop(bp[bp.SYSTOLIC_BLOOD_PRESSURE == 'ERROR'].index, inplace=True)
bp.sort_values('SYSTOLIC_BLOOD_PRESSURE', ascending=False).head()

,PERSON_ID,TIME,SYSTOLIC_BLOOD_PRESSURE
357,234,9/8/2015 9:53,99
350,234,9/8/2015 9:46,99
311,234,9/8/2015 9:07,99
1187,456,8/15/2015 10:03,99
1001,456,6/8/2015 10:05,99


In [107]:
bp = bp.dropna()
bd = bp.drop(bp[bp.SYSTOLIC_BLOOD_PRESSURE == 'ERROR'].index, inplace=True)
bp['SERVICE_DATE'] = bp['TIME'].apply(lambda x: x.split(' ')[0])
bp['TIME'] = bp['TIME'].apply(lambda x: x.split(' ')[1])
bp.sort_values('SYSTOLIC_BLOOD_PRESSURE', ascending=False).head()

,PERSON_ID,TIME,SYSTOLIC_BLOOD_PRESSURE,SERVICE_DATE
357,234,9:53,99,9/8/2015
350,234,9:46,99,9/8/2015
311,234,9:07,99,9/8/2015
1187,456,10:03,99,8/15/2015
1001,456,10:05,99,6/8/2015


## Merging the Dataframes

I'll join the dataframes on the PERSON_ID column and then on the SERVICE_DATE column.

In [108]:
df = pd.merge(bp, dems, on=['PERSON_ID', 'SERVICE_DATE'], how='left')
df.head()

,PERSON_ID,TIME,SYSTOLIC_BLOOD_PRESSURE,SERVICE_DATE,AGE_MONTHS
0,123,7:16,33,1/5/2016,46.0
1,123,7:17,75,1/5/2016,46.0
2,123,7:18,58,1/5/2016,46.0
3,123,7:19,93,1/5/2016,46.0
4,123,7:20,35,1/5/2016,46.0


In [109]:
print('BloodPressure length: ', len(bp))
print('Merged DF length: ', len(df))

BloodPressure length:  1576
Merged DF length:  1576


In [110]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1576 entries, 0 to 1575
Data columns (total 5 columns):
PERSON_ID                  1576 non-null int64
TIME                       1576 non-null object
SYSTOLIC_BLOOD_PRESSURE    1576 non-null object
SERVICE_DATE               1576 non-null object
AGE_MONTHS                 1540 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 73.9+ KB


,PERSON_ID,AGE_MONTHS
count,1576.000000,1540.000000
mean,376.583756,44.015584
std,211.955585,2.934816
min,111.000000,39.000000
25%,234.000000,42.000000
50%,345.000000,44.000000
75%,456.000000,47.000000
max,987.000000,50.000000


In [111]:
df.AGE_MONTHS.fillna(46.0, inplace=True)

In [112]:
bp.SYSTOLIC_BLOOD_PRESSURE.value_counts()

88     37
33     36
44     35
73     35
100    34
       ..
82     13
79      9
76      9
28      2
26      1
Name: SYSTOLIC_BLOOD_PRESSURE, Length: 74, dtype: int64

In [113]:
# converting blood pressure to float from object
df['SYSTOLIC_BLOOD_PRESSURE'] = df['SYSTOLIC_BLOOD_PRESSURE'].astype(float)

## Identifying the Cases 

I've created a function that takes a dataframe as input and takes the age of a patient and uses this to compare whether the blood pressure was above or below a predetermined normal on a given surgery date. 

In [114]:

def extended_blood_pressure(df):
    # separating the patients by ID
    patients = df.PERSON_ID.unique()
    for patient in patients:
        patient_df = df[df['PERSON_ID'] == patient].reset_index() # dataframe for each individual patient
        dates = patient_df.SERVICE_DATE.unique() # list of surgery dates for each patient
        
        # separating the surgery dates
        for date in dates: 
            surgeries = patient_df[patient_df['SERVICE_DATE'] == date].reset_index() # dataframe for each surgery for each individual patient
            age = surgeries.iloc[0]['AGE_MONTHS']
            
            # comparing the blood pressure
            continuous_min_below_norm = 0
            for i, row in surgeries.iterrows():
                blood_pressure = surgeries.iloc[i]['SYSTOLIC_BLOOD_PRESSURE']
                
                # checking children under 44 months
                if age < 44.0:
                    if blood_pressure <= 46.0:
                
                        continuous_min_below_norm += 1
                    elif blood_pressure >  46.0 and continuous_min_below_norm >= 14:
                        print('Patient {} dropped below the norm for {} minutes on {}.'.format(surgeries.iloc[i]['PERSON_ID'], 
                                                                                               continuous_min_below_norm, surgeries.iloc[i]['SERVICE_DATE']))
                        continuous_min_below_norm = 0
                    else: 
                        continuous_min_below_norm = 0 
                    
                # checking children over 44 months
                else:
                    if blood_pressure <= 55.0:
                        continuous_min_below_norm += 1
                    elif blood_pressure >=  55.0 and continuous_min_below_norm >= 14:
                        print('Patient {} dropped below the norm for {} minutes on {}.'.format(surgeries.iloc[i]['PERSON_ID'], 
                                                                                               continuous_min_below_norm, surgeries.iloc[i]['SERVICE_DATE']))
                        continuous_min_below_norm = 0
                    else: 
                        continuous_min_below_norm = 0 


In [117]:
extended_blood_pressure(df)

Patient 123 dropped below the norm for 14 minutes on 1/5/2016.
Patient 456 dropped below the norm for 20 minutes on 5/21/2015.
Patient 789 dropped below the norm for 16 minutes on 7/7/2017.
Patient 987 dropped below the norm for 16 minutes on 2/17/2016.
Patient 987 dropped below the norm for 24 minutes on 10/17/2016.
Patient 111 dropped below the norm for 33 minutes on 8/15/2015.


This function does exactly the same, and in addition creates a file called Perides_Interview for the report.

In [115]:

def extended_blood_pressure_file(df):
    file = open("Perides_Interview.txt", "w")
    # separating the patients by ID
    patients = df.PERSON_ID.unique()
    for patient in patients:
        patient_df = df[df['PERSON_ID'] == patient].reset_index() # dataframe for each individual patient
        dates = patient_df.SERVICE_DATE.unique() # list of surgery dates for each patient
        
        # separating the surgery dates
        for date in dates: 
            surgeries = patient_df[patient_df['SERVICE_DATE'] == date].reset_index() # dataframe for each surgery for each individual patient
            age = surgeries.iloc[0]['AGE_MONTHS']
            
            # comparing the blood pressure
            continuous_min_below_norm = 0
            for i, row in surgeries.iterrows():
                blood_pressure = surgeries.iloc[i]['SYSTOLIC_BLOOD_PRESSURE']
                
                # checking children under 44 months
                if age < 44.0:
                    if blood_pressure <= 46.0:
                
                        continuous_min_below_norm += 1
                    elif blood_pressure >  46.0 and continuous_min_below_norm >= 14:
                        print('Patient {} dropped below the norm for {} minutes on {}.'.format(surgeries.iloc[i]['PERSON_ID'], 
                                                                                               continuous_min_below_norm, surgeries.iloc[i]['SERVICE_DATE']))
                        file.write('Patient {} dropped below the norm for {} minutes on {}.\n'.format(surgeries.iloc[i]['PERSON_ID'], 
                                                                                               continuous_min_below_norm, surgeries.iloc[i]['SERVICE_DATE']))
                        continuous_min_below_norm = 0
                    else: 
                        continuous_min_below_norm = 0 
                    
                # checking children over 44 months
                else:
                    if blood_pressure <= 55.0:
                        continuous_min_below_norm += 1
                    elif blood_pressure >=  55.0 and continuous_min_below_norm >= 14:
                        print('Patient {} dropped below the norm for {} minutes on {}.'.format(surgeries.iloc[i]['PERSON_ID'], 
                                                                                               continuous_min_below_norm, surgeries.iloc[i]['SERVICE_DATE']))
                        file.write('Patient {} dropped below the norm for {} minutes on {}.\n'.format(surgeries.iloc[i]['PERSON_ID'], 
                                                                                               continuous_min_below_norm, surgeries.iloc[i]['SERVICE_DATE']))
                        continuous_min_below_norm = 0
                    else: 
                        continuous_min_below_norm = 0 


In [116]:
extended_blood_pressure_file(df)

Patient 123 dropped below the norm for 14 minutes on 1/5/2016.
Patient 456 dropped below the norm for 20 minutes on 5/21/2015.
Patient 789 dropped below the norm for 16 minutes on 7/7/2017.
Patient 987 dropped below the norm for 16 minutes on 2/17/2016.
Patient 987 dropped below the norm for 24 minutes on 10/17/2016.
Patient 111 dropped below the norm for 33 minutes on 8/15/2015.
